In [431]:
import pandas as pd
import yfinance as yf

#dfSPY = pd.read_csv("SPY.USUSD_Candlestick_1_D_BID_16.02.2017-21.05.2022.csv")
#dfSPY=yf.download("^GSPC",start='2011-01-05', end='2021-01-05')
dfSPY=yf.download("^RUI",start='2011-01-05', end='2021-01-05')
#dfSPY=yf.download("EURUSD=X",start='2011-01-05', end='2021-01-05')

dfSPY=dfSPY[dfSPY.High!=dfSPY.Low]
dfSPY.reset_index(inplace=True)
dfSPY.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-01-05,704.739990,707.419983,699.049988,706.869995,706.869995,0
1,2011-01-06,702.380005,707.929993,700.679993,705.479980,705.479980,0
2,2011-01-07,706.830017,707.929993,698.780029,704.320007,704.320007,0
3,2011-01-10,703.229980,704.770020,699.020020,703.760010,703.760010,0
4,2011-01-11,705.090027,707.929993,703.859985,706.520020,706.520020,0


In [432]:
import pandas_ta as ta
dfSPY['EMA']=ta.sma(dfSPY.Close, length=200)#sma ema
dfSPY['RSI']=ta.rsi(dfSPY.Close, length=2)
#dfSPY.ta.indicators()
#help(ta.bbands)
my_bbands = ta.bbands(dfSPY.Close, length=20, std=2.5)
my_bbands[0:50]
dfSPY=dfSPY.join(my_bbands)
dfSPY.dropna(inplace=True)
dfSPY.reset_index(inplace=True)
dfSPY

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5
0,199,2011-10-19,674.260010,679.320007,665.030029,666.960022,666.960022,0,707.434350,41.559158,595.548746,644.963004,694.377261,15.323129,0.722578
1,200,2011-10-20,667.630005,672.059998,659.520020,669.909973,669.909973,0,707.249550,54.360857,597.704805,647.285004,696.865202,15.319434,0.728165
2,201,2011-10-21,672.059998,683.219971,672.059998,682.700012,682.700012,0,707.135650,84.259625,598.263168,650.042004,701.820841,15.930920,0.815361
3,202,2011-10-24,683.280029,693.890015,683.280029,692.719971,692.719971,0,707.077650,92.232556,596.124520,652.589502,709.054484,17.304901,0.855357
4,203,2011-10-25,691.380005,691.380005,677.400024,678.590027,678.590027,0,706.951800,37.977830,595.911512,654.066504,712.221496,17.782593,0.710846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2311,2510,2020-12-28,2104.199951,2114.870117,2104.199951,2111.629883,2111.629883,138600,1786.428447,95.690101,2033.935674,2079.879010,2125.822346,4.417885,0.845544
2312,2511,2020-12-29,2112.600098,2123.340088,2102.620117,2105.449951,2105.449951,173400,1790.423146,55.957856,2042.323460,2083.283508,2124.243557,3.932259,0.770586
2313,2512,2020-12-30,2107.959961,2116.459961,2107.959961,2109.719971,2109.719971,127000,1794.064296,72.015218,2045.005397,2085.833508,2126.661620,3.914801,0.792525
2314,2513,2020-12-31,2109.669922,2122.909912,2105.919922,2120.870117,2120.870117,53400,1798.145847,90.363651,2046.366396,2088.824512,2131.282628,4.065264,0.877379


In [433]:
def addemasignal(df, backcandles):
    emasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.EMA[i]:
                dnt=0
            if df.Low[i]<=df.EMA[i]:
                upt=0
        if upt==1 and dnt==1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row]=3
        elif upt==1:
            emasignal[row]=2
        elif dnt==1:
            emasignal[row]=1
    df['EMASignal'] = emasignal

addemasignal(dfSPY, 6)

In [434]:
def addorderslimit(df, percent):
    ordersignal=[0]*len(df)
    for i in range(1, len(df)): #EMASignal of previous candle!!! modified!!!
        if df.EMASignal[i]==2 and df.Close[i]<=df['BBL_20_2.5'][i]:# and df.RSI[i]<=100: #Added RSI condition to avoid direct close condition
            ordersignal[i]=df.Close[i]-df.Close[i]*percent
        elif df.EMASignal[i]==1 and df.Close[i]>=df['BBU_20_2.5'][i]:# and df.RSI[i]>=0:
            ordersignal[i]=df.Close[i]+df.Close[i]*percent
    df['ordersignal']=ordersignal
    
addorderslimit(dfSPY, 0.00)

In [435]:
dfSPY[dfSPY.ordersignal!=0]

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5,EMASignal,ordersignal
118,317,2012-04-10,763.659973,764.729980,750.030029,750.619995,750.619995,0,702.808600,0.953716,756.865835,774.960501,793.055167,4.669829,-0.172588,2,750.619995
418,617,2013-06-20,902.469971,902.469971,877.760010,879.830017,879.830017,0,833.538099,9.250886,881.826258,905.800000,929.773742,5.293385,-0.041634,2,879.830017
457,656,2013-08-15,937.030029,937.030029,922.070007,923.619995,923.619995,0,858.857550,3.584613,925.725225,940.277509,954.829792,3.095317,-0.072333,2,923.619995
458,657,2013-08-16,923.580017,925.119995,919.150024,920.669983,920.669983,0,859.567550,2.661574,921.308228,939.367007,957.425787,3.844883,-0.017671,2,920.669983
459,658,2013-08-19,920.619995,922.460022,915.010010,915.109985,915.109985,0,860.246350,1.350614,915.740900,938.071005,960.401110,4.760856,-0.014127,2,915.109985
494,693,2013-10-08,936.729980,937.250000,924.489990,924.559998,924.559998,0,887.705700,9.961421,925.276037,945.320999,965.365962,4.240880,-0.017861,2,924.559998
568,767,2014-01-24,1019.880005,1021.739990,999.929993,999.929993,999.929993,0,946.441652,3.019828,1006.834802,1024.323505,1041.812208,3.414684,-0.197408,2,999.929993
569,768,2014-01-27,1000.390015,1003.020020,989.359985,994.500000,994.500000,0,946.992952,2.164061,998.925975,1022.717505,1046.509034,4.652610,-0.093016,2,994.500000
574,773,2014-02-03,996.570007,997.479980,971.640015,972.820007,972.820007,0,950.157451,10.459400,973.315595,1014.675000,1056.034405,8.152247,-0.005991,2,972.820007
622,821,2014-04-11,1019.820007,1024.250000,1012.140015,1013.010010,1013.010010,0,982.207202,13.522048,1014.039786,1040.303998,1066.568210,5.049334,-0.019604,2,1013.010010


# Visualization

In [436]:
import numpy as np
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
dfSPY['pointposbreak'] = dfSPY.apply(lambda row: pointposbreak(row), axis=1)

In [437]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = dfSPY[1000:1250].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_20_2.5'], line=dict(color='blue', width=1), name="BBL_20_2.5"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_20_2.5'], line=dict(color='blue', width=1), name="BBU_20_2.5")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [438]:
dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [439]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    ordertime=[]
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        
        for j in range(0, len(self.orders)):
            #print('!!!!!!!!!!!!!!!!!!!', self.data.index[-1])
            if self.data.index[-1]-self.ordertime[0]>5:#days max to fulfill the order!!!
                #print('----------------------')
                #print(self.orders)
                #print(self.ordertime)
                self.orders[0].cancel()
                self.ordertime.pop(0)   
            
        if len(self.trades)>0:
            #print(self.data.index[-1], self.trades)
            if self.data.index[-1]-self.trades[-1].entry_time>=10:
                self.trades[-1].close()
                #print(self.data.index[-1], self.trades[-1].entry_time)
            
            if self.trades[-1].is_long and self.data.RSI[-1]>=50:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=50:
                self.trades[-1].close()
        
        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.buy(sl=self.signal/2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.sell(sl=self.signal*2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/10, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    2315.0
Duration                               2315.0
Exposure Time [%]                     3.88601
Equity Final [$]                 57917.223389
Equity Peak [$]                  61507.581177
Return [%]                         479.172234
Buy & Hold Return [%]              213.320124
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -56.198819
Avg. Drawdown [%]                   -9.409611
Max. Drawdown Duration                  417.0
Avg. Drawdown Duration              73.333333
# Trades                                 23.0
Win Rate [%]                        86.956522
Best Trade [%]                       2.195045
Worst Trade [%]                     -1.453326
Avg. Trade [%]                    

In [440]:
bt.plot(show_legend=False)

Row(id='31681', ...)

In [441]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,112,419,422,879.820007,879.820007,0.000000,0.000000,419,422,3
1,107,458,463,923.580017,922.320007,-134.821045,-0.001364,458,463,5
2,105,495,497,924.489990,943.599976,2006.548462,0.020671,495,497,2
3,117,569,573,999.929993,1001.299988,160.289429,0.001370,569,573,4
4,117,623,625,1013.010010,1030.430054,2038.145142,0.017196,623,625,2
5,129,699,701,1074.089966,1077.959961,499.229370,0.003603,699,701,2
6,133,742,744,1081.380005,1096.089966,1956.424805,0.013603,742,744,2
7,147,793,796,1111.900024,1124.619995,1869.835693,0.011440,793,796,3
8,159,850,852,1141.060059,1152.540039,1825.316895,0.010061,850,852,2
9,174,927,929,1149.199951,1162.630005,2336.829346,0.011686,927,929,2
